<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Setup" data-toc-modified-id="Setup-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Setup</a></span></li><li><span><a href="#Introduction" data-toc-modified-id="Introduction-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Introduction</a></span></li><li><span><a href="#One-dimensional-examples" data-toc-modified-id="One-dimensional-examples-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>One-dimensional examples</a></span><ul class="toc-item"><li><span><a href="#Basic-setup" data-toc-modified-id="Basic-setup-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Basic setup</a></span></li><li><span><a href="#Single-point" data-toc-modified-id="Single-point-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Single point</a></span></li><li><span><a href="#Vector-of-points" data-toc-modified-id="Vector-of-points-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Vector of points</a></span></li><li><span><a href="#Multi-function-interpolation-(point)" data-toc-modified-id="Multi-function-interpolation-(point)-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Multi-function interpolation (point)</a></span></li><li><span><a href="#Multi-function-interpolation-(vector)" data-toc-modified-id="Multi-function-interpolation-(vector)-3.5"><span class="toc-item-num">3.5&nbsp;&nbsp;</span>Multi-function interpolation (vector)</a></span></li></ul></li><li><span><a href="#Multi-dimensional-examples" data-toc-modified-id="Multi-dimensional-examples-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Multi-dimensional examples</a></span><ul class="toc-item"><li><span><a href="#Without-monotonoicity" data-toc-modified-id="Without-monotonoicity-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Without monotonoicity</a></span></li><li><span><a href="#With-monotonicity" data-toc-modified-id="With-monotonicity-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>With monotonicity</a></span></li></ul></li><li><span><a href="#Parallel" data-toc-modified-id="Parallel-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Parallel</a></span></li><li><span><a href="#Speed-Tests-(without-parallization)" data-toc-modified-id="Speed-Tests-(without-parallization)-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Speed Tests (without parallization)</a></span><ul class="toc-item"><li><span><a href="#Loop-with-interpolation-for-single-point" data-toc-modified-id="Loop-with-interpolation-for-single-point-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>Loop with interpolation for single point</a></span></li><li><span><a href="#Interpolation-for-a-vector-of-points" data-toc-modified-id="Interpolation-for-a-vector-of-points-6.2"><span class="toc-item-num">6.2&nbsp;&nbsp;</span>Interpolation for a vector of points</a></span></li></ul></li></ul></div>

**Abstract:** This notebooks shows how to use the **linear_interp** module.

# Setup

In [1]:
# imports and settings
%matplotlib inline
%load_ext autoreload
%autoreload 1

# imports
import time
import numba
import numpy as np
from vfi import linear_interp
from vfi import linear_interp_test
%aimport vfi

np.random.seed(2018)

In [2]:
%%html
<style>
.output_wrapper, .output {
    height:auto !important;
    max-height:5000px;  /* your desired max-height here */
}
.output_scroll {
    box-shadow:none !important;
    webkit-box-shadow:none !important;
}
</style>

# Introduction

The **linear_interp** module provides a **Numba JIT** compilled **interpolator class** for **linear interpolation** (and extrapolation) of **multiple functions** in **n-dimensions** for:

1. A *single* point
2. A *vector* of points
3. A *vector* of *monotonic* points, where the input array can be written

$$ x = [[x_1,x_2,...,x_{d-1},x_d^1],[x_1,x_2,...,x_{d-1},x_d^2],...,[x_1,x_2,...,x_{d-1},x_d^n]]$$
$$ \text{ where } x_d^1 < x_d^2 < ... < x_d^n $$


The interpolator class can be used inside a parallel loop (**prange**, see section 5).

** Speed tests without parallization: ** 

Using a test case with **3 input dimensions** and **2 output dimensions** the speed-up relative to **scipy**'s **RegularGridInterpolator** is:

1. \> 40 for a long loop with interpolation of a single point
2. \> 3 for a vector of points
3. \> 10 for a vector of *monotonic* points

# One-dimensional examples

In this section, we show some one-dimensional examples of how to use the interpolator class.

## Basic setup

In [3]:
# a. function
def f(x):
    return x*x

# b. grid pints
low_x = 0
high_x = 10
Nx = 50
grid_x = np.linspace(low_x,high_x,Nx)

# c. known values
ff = f(grid_x)

# d. unknown points
Nxi = 100
xi = np.random.uniform(low=low_x,high=1.1*high_x,size=(Nxi,1))

## Single point

In [4]:
# a. interpolator
fhat = linear_interp.create_interpolator(grids=[grid_x],values=[ff],Nxi=1)

# b. interpolate
for i in range(5):
    fhat_xi = fhat.evaluate(xi[i,:])[0]
    f_xi = f(xi[i,0])
    print('x = {:5.2f} -> fhat = {:6.2f} (true {:6.2f})'.format(xi[i,0],fhat_xi,f_xi))

x =  9.71 -> fhat =  94.21 (true  94.20)
x =  1.15 -> fhat =   1.33 (true   1.32)
x =  9.98 -> fhat =  99.55 (true  99.54)
x =  3.37 -> fhat =  11.37 (true  11.36)
x =  4.91 -> fhat =  24.12 (true  24.11)


## Vector of points

In [5]:
# a. interpolator
fhat = linear_interp.create_interpolator(grids=[grid_x],values=[ff],Nxi=Nxi)

# b. interpolate
fhat_xi_vec = fhat.evaluate(xi.ravel())
for i in range(5):
    f_xi = f(xi[i,0])
    print('x = {:5.2f} -> fhat = {:6.2f} (true {:6.2f})'.format(xi[i,0],fhat_xi_vec[i],f_xi))

x =  9.71 -> fhat =  94.21 (true  94.20)
x =  1.15 -> fhat =   1.33 (true   1.32)
x =  9.98 -> fhat =  99.55 (true  99.54)
x =  3.37 -> fhat =  11.37 (true  11.36)
x =  4.91 -> fhat =  24.12 (true  24.11)


## Multi-function interpolation (point)

In [6]:
# a. new function
def g(x):
    return x*x - 5

# b. known values (same grid)
gg = g(grid_x)

# c. interpolator
fghat = linear_interp.create_interpolator(grids=[grid_x],values=[ff,gg],Nxi=1)

# d. interpolate
for i in range(5):
    fghat_xi = fghat.evaluate(xi[i,:])
    f_xi = f(xi[i,0])
    g_xi = g(xi[i,0])
    print('x = {:5.2f} -> fhat = {:6.2f} (true {:6.2f}), ghat = {:6.2f} (true {:6.2f})'.format(
        xi[i,0],fghat_xi[0],f_xi,fghat_xi[1],g_xi))

x =  9.71 -> fhat =  94.21 (true  94.20), ghat =  89.21 (true  89.20)
x =  1.15 -> fhat =   1.33 (true   1.32), ghat =  -3.67 (true  -3.68)
x =  9.98 -> fhat =  99.55 (true  99.54), ghat =  94.55 (true  94.54)
x =  3.37 -> fhat =  11.37 (true  11.36), ghat =   6.37 (true   6.36)
x =  4.91 -> fhat =  24.12 (true  24.11), ghat =  19.12 (true  19.11)


## Multi-function interpolation (vector)

In [7]:
# a. interpolator
fghat = linear_interp.create_interpolator(grids=[grid_x],values=[ff,gg],Nxi=Nxi)

# b. interpolate
fghat_xis = fghat.evaluate(xi.ravel())
for i in range(5): 
    f_xi = f(xi[i,0])
    g_xi = g(xi[i,0])
    print('x = {:5.2f} -> fhat = {:6.2f} (true f = {:6.2f}), ghat = {:6.2f} (true g = {:6.2f})'.format(
        xi[i,0],fghat_xis[i],f_xi,fghat_xis[Nxi+i],g_xi))

x =  9.71 -> fhat =  94.21 (true f =  94.20), ghat =  89.21 (true g =  89.20)
x =  1.15 -> fhat =   1.33 (true f =   1.32), ghat =  -3.67 (true g =  -3.68)
x =  9.98 -> fhat =  99.55 (true f =  99.54), ghat =  94.55 (true g =  94.54)
x =  3.37 -> fhat =  11.37 (true f =  11.36), ghat =   6.37 (true g =   6.36)
x =  4.91 -> fhat =  24.12 (true f =  24.11), ghat =  19.12 (true g =  19.11)


# Multi-dimensional examples

In this section, we show some multi-dimensional examples of how to use the interpolator class.

## Without monotonoicity

In [8]:
# a. functions
def f(x,y):
    return x*x + y

def g(x,y):
    return x*x + y - 5

# b. grid pints
low_x = 0
low_y = 0
high_x = 10
high_y = 8
Nx = 50
Ny = 50
grid_x = np.linspace(low_x,high_x,Nx)
grid_y = np.linspace(low_y,high_y,Ny)

# c. known values
xx, yy = np.meshgrid(grid_x,grid_y,indexing='ij')
ff = f(xx,yy)
gg = g(xx,yy)

# d. unknown points
Nxi = 100
xi = np.zeros((Nxi,2))
xi[:,0] = np.random.uniform(low=low_x,high=1.1*high_x,size=(Nxi,))
xi[:,1] = np.random.uniform(low=low_y,high=1.1*high_y,size=(Nxi,))

# e. interpolator
fghat = linear_interp.create_interpolator(grids=[grid_x,grid_y],values=[ff,gg],Nxi=Nxi)

# f. interpolate
fghat_xis = fghat.evaluate(xi.ravel())
for i in range(5): 
    f_xi = f(xi[i,0],xi[i,1])
    g_xi = g(xi[i,0],xi[i,1])
    print('(x,y) = ({:5.2f},{:5.2f}) -> fhat = {:6.2f} (true f = {:6.2f}), ghat = {:6.2f} (true g = {:6.2f})'.format(
        xi[i,0],xi[i,1],fghat_xis[i],f_xi,fghat_xis[Nxi+i],g_xi))

(x,y) = ( 5.84, 2.49) -> fhat =  36.62 (true f =  36.61), ghat =  31.62 (true g =  31.61)
(x,y) = ( 4.95, 3.84) -> fhat =  28.39 (true f =  28.38), ghat =  23.39 (true g =  23.38)
(x,y) = ( 7.33, 3.95) -> fhat =  57.73 (true f =  57.73), ghat =  52.73 (true g =  52.73)
(x,y) = ( 6.21, 5.73) -> fhat =  44.24 (true f =  44.23), ghat =  39.24 (true g =  39.23)
(x,y) = ( 0.06, 7.83) -> fhat =   7.84 (true f =   7.84), ghat =   2.84 (true g =   2.84)


## With monotonicity

In [9]:
# g. unknown points with monotonicity
xi_vec = np.linspace(low_y,1.2*high_y,Nxi)

xi_base = np.zeros(2)
xi_base[0:1] = np.random.uniform(low=low_x,high=1.1*high_x,size=(1,))
xi_base[1] = xi_vec[0]

xi = np.zeros((Nxi,2))
xi[:,0:1] = xi_base[0:1]
xi[:,1] = xi_vec
  
# h. interpolate
fghat_xis = fghat.evaluate_monotone(xi_base,xi_vec)
for i in range(10): 
    f_xi = f(xi[i,0],xi[i,1])
    g_xi = g(xi[i,0],xi[i,1])
    print('(x,y) = ({:5.2f},{:5.2f}) -> fhat = {:6.2f} (true f = {:6.2f}), ghat = {:6.2f} (true g = {:6.2f})'.format(
        xi[i,0],xi[i,1],fghat_xis[i],f_xi,fghat_xis[Nxi+i],g_xi))

(x,y) = ( 0.57, 0.00) -> fhat =   0.33 (true f =   0.32), ghat =  -4.67 (true g =  -4.68)
(x,y) = ( 0.57, 0.10) -> fhat =   0.43 (true f =   0.42), ghat =  -4.57 (true g =  -4.58)
(x,y) = ( 0.57, 0.19) -> fhat =   0.52 (true f =   0.52), ghat =  -4.48 (true g =  -4.48)
(x,y) = ( 0.57, 0.29) -> fhat =   0.62 (true f =   0.61), ghat =  -4.38 (true g =  -4.39)
(x,y) = ( 0.57, 0.39) -> fhat =   0.72 (true f =   0.71), ghat =  -4.28 (true g =  -4.29)
(x,y) = ( 0.57, 0.48) -> fhat =   0.81 (true f =   0.81), ghat =  -4.19 (true g =  -4.19)
(x,y) = ( 0.57, 0.58) -> fhat =   0.91 (true f =   0.90), ghat =  -4.09 (true g =  -4.10)
(x,y) = ( 0.57, 0.68) -> fhat =   1.01 (true f =   1.00), ghat =  -3.99 (true g =  -4.00)
(x,y) = ( 0.57, 0.78) -> fhat =   1.10 (true f =   1.10), ghat =  -3.90 (true g =  -3.90)
(x,y) = ( 0.57, 0.87) -> fhat =   1.20 (true f =   1.19), ghat =  -3.80 (true g =  -3.81)


# Parallel

In this section, we show how to use the interpolator class inside a parallel loop.

In [10]:
# a. functions
def f(x,y):
    return x*x + y

# b. grid pints
Nx = 100
grid_x = np.linspace(0,1,Nx)

# c. known values
xx, yy = np.meshgrid(grid_x,grid_y,indexing='ij')
ff = f(xx,yy)

# d. unknown points
Nxi = 10**6
xi = np.random.uniform(size=(Nxi,2))

# e. test function
@numba.njit(nogil=True)    
def test_serial(x,dimx,Nx,y,dimy,Ny,Nxi,xi,yi,long_length):    
    for i in range(long_length):
        interpolator = linear_interp.interpolator(x,dimx,Nx,y,dimy,Ny,Nxi) # jitted create
        yi[i*Nxi:(i+1)*Nxi] = interpolator.evaluate(xi)
    
@numba.njit(parallel=True) 
def test_parallel(x,dimx,Nx,y,dimy,Ny,Nxi,xi,yi,long_length):    
    for i in numba.prange(long_length):        
        interpolator = linear_interp.interpolator(x,dimx,Nx,y,dimy,Ny,Nxi) # jitted create
        yi[i*Nxi:(i+1)*Nxi] = interpolator.evaluate(xi)

# f. needed information 
s = linear_interp.create_interpolator_dict(grids=[grid_x,grid_y],values=[ff],Nxi=Nxi)

loop_length = 40
yi_serial = np.zeros((loop_length*Nxi,1))
yi_parallel = np.zeros((loop_length*Nxi,1))

# g. run
tic = time.time()
test_serial(s.x,s.dimx,s.Nx,s.y,s.dimy,s.Ny,s.Nxi,xi.ravel(),yi_serial.ravel(),loop_length)
toc = time.time()
print(f'serial: \t {toc-tic:.1f} secs')

tic = time.time()
test_parallel(s.x,s.dimx,s.Nx,s.y,s.dimy,s.Ny,s.Nxi,xi.ravel(),yi_parallel.ravel(),loop_length)
toc = time.time()
print(f'parallel: \t {toc-tic:.1f} secs')

assert np.allclose(yi_serial,yi_parallel)

serial: 	 10.0 secs
parallel: 	 4.0 secs


# Speed Tests (without parallization)

## Loop with interpolation for single point

In this sub-section we showsbest of 5 timings for a test case with **3 input dimensions** and **2 output dimensions** and a long loop with interpolation of a single point.

In [11]:
for loop_length in [10**3,5*10**3,10**4]:
    linear_interp_test.single(loop_length)

loop length 	 1000
linear_interp 	 0.00 secs
scipy 		 0.20 secs
speed-up 	 65.7

loop length 	 5000
linear_interp 	 0.02 secs
scipy 		 1.01 secs
speed-up 	 55.8

loop length 	 10000
linear_interp 	 0.04 secs
scipy 		 1.93 secs
speed-up 	 43.8



## Interpolation for a vector of points

In this sub-section we show best of 5 timings for a test case with **3 input dimensions** and **2 output dimensions** and interpolation of a large vector of points.

In [12]:
for Nxi in [5*10**5,10**6,5*10**6]:
    linear_interp_test.vector(Nxi)

vector length 	 500000
linear_interp 	 0.05 secs
linear_interp 	 0.15 secs (no monotonicity)
scipy 		 0.82 secs
speed-up 	 18.2
speed-up 	 5.4 (no monotonicity)

vector length 	 1000000
linear_interp 	 0.09 secs
linear_interp 	 0.30 secs (no monotonicity)
scipy 		 1.68 secs
speed-up 	 19.0
speed-up 	 5.6 (no monotonicity)

vector length 	 5000000
linear_interp 	 0.43 secs
linear_interp 	 1.52 secs (no monotonicity)
scipy 		 8.95 secs
speed-up 	 20.6
speed-up 	 5.9 (no monotonicity)

